# Generate GRN tutorial

## Initialization
- We loaded necessary R packages which were used for the data analysis.

In [2]:
library('devtools')
library('scTenifoldNet')
library('stats')
library('Matrix')

## Loading data
- Use normalized gene activities as a matrix (gene by cell format)
- The sample rds data can be downloaded from the download page

In [16]:
wt_mat <- readRDS(file = "../norm_WT_gene_activity_by_cell.rds") 

## scTenifoldKnk core code is extracted to generate GRN

In [18]:
# Conditional filtering
qFilter <- function(X, q = 0){
  X <- as.matrix(X)
  X[abs(X) < stats::quantile(abs(X),0.95)] <- 0
  X <- Matrix::Matrix(X)
  return(X)
}

# QC quality control
scQC <- function(X, mtThreshold = 0.1, minLSize = 1000){
  if(class(X) == 'Seurat'){
    countMatrix <- X@assays$RNA@counts
  } else {
    countMatrix <- X
  }
  librarySize <- colSums(countMatrix)
  countMatrix <- countMatrix[,librarySize >= minLSize]
  librarySize <- colSums(countMatrix)
  mtGenes <- grep('^MT-',toupper(rownames(countMatrix)))
  nGenes <- colSums(countMatrix != 0)

  genesLM <- lm(nGenes~librarySize)
  genesLM <- as.data.frame(predict(genesLM, data.frame(librarySize), interval = 'prediction'))

  if(isTRUE(length(mtGenes) > 0)){
    mtCounts <- colSums(countMatrix[grep('^MT-',toupper(rownames(countMatrix))),])
    mtProportion <- mtCounts/librarySize
    mtLM <- lm(mtCounts~librarySize)
    mtLM <- as.data.frame(predict(mtLM, data.frame(librarySize), interval = 'prediction'))
    selectedCells <- mtCounts > mtLM$lwr & mtCounts < mtLM$upr & nGenes > genesLM$lwr & nGenes < genesLM$upr & mtProportion <= mtThreshold & librarySize < 2 * mean(librarySize)
  } else {
    selectedCells <- nGenes > genesLM$lwr & nGenes < genesLM$upr & librarySize < 2 * mean(librarySize)
  }
  selectedCells <- colnames(countMatrix)[selectedCells]
  if(class(X) == 'Seurat'){
    X <- subset(X, cells = selectedCells)
  } else {
    X <- countMatrix[,selectedCells]
  }
  return(X)
}

# Strictly directed graph 
strictDirection <- function(X, lambda = 1){
  S <- as.matrix(X)
  S[abs(S) < abs(t(S))] <- 0
  O <- (((1-lambda) * X) + (lambda * S))
  O <- Matrix::Matrix(O)
  return(O)
}

# main 
scTenifoldKnk <- function(countMatrix, qc = TRUE, gKO = NULL, qc_mtThreshold = 0.1, qc_minLSize = 1000, nc_lambda = 0, nc_nNet = 10, nc_nCells = 500, nc_nComp = 3,
                          nc_scaleScores = TRUE, nc_symmetric = FALSE, nc_q = 0.9, td_K = 3, td_maxIter = 1000,
                          td_maxError = 1e-05, td_nDecimal = 3, ma_nDim = 2, nCores = 64){

  if(isTRUE(qc)){
    countMatrix <- scQC(countMatrix, mtThreshold = qc_mtThreshold, minLSize = qc_minLSize)
  }

  if(ncol(countMatrix) > 500){
    countMatrix <- countMatrix[rowMeans(countMatrix != 0) >= 0.05,] # Adjust this parameter dynamically
  } else {
    countMatrix[rowSums(countMatrix != 0) >= 25,]
  }
  WT <- scTenifoldNet::makeNetworks(X = countMatrix, q = nc_q, nNet = nc_nNet, nCells = nc_nCells, scaleScores = nc_scaleScores, symmetric = nc_symmetric, nComp = nc_nComp, nCores = nCores)

  # Tensor decomposition
  WT <- scTenifoldNet::tensorDecomposition(xList = WT, K = td_K, maxError = td_maxError, maxIter = td_maxIter, nDecimal = td_nDecimal)

  WT <- WT$X
  WT <- strictDirection(WT, lambda = nc_lambda)
  WT <- as.matrix(WT)
  diag(WT) <- 0
  WT <- t(WT)
    
  # Preparing output
  outputList <- list()
  outputList$tensorNetworks$WT <- Matrix(WT)
    
  return(outputList)
}



In [19]:
norm_pcnetOutput <- scTenifoldKnk(countMatrix = wt_mat, 
                             nc_lambda = 1, # Strictly directed graphs
                             nc_q = 0.9, # Keep the core
                             nc_scaleScores = FALSE, # Unnormalized weights
                             td_nDecimal = 8, # Keep decimals
                             nCores = 12,
                             qc_minLSize = 1000) 

norm_pcnetOutput = norm_pcnetOutput$tensorNetworks$WT

In [ ]:
# Lexicographic sorting
pcnetOutput_ordered = norm_pcnetOutput[order(rownames(norm_pcnetOutput)), order(colnames(norm_pcnetOutput))]

## Save the GRN as a mm file

In [ ]:
pcnetOutput_ordered_sp <- as(pcnetOutput_ordered, "sparseMatrix")

writeMM(pcnetOutput_ordered_sp, "./mm10_scATAC_GRN.mm")

## Save GRN as an rds file

In [ ]:
saveRDS(pcnetOutput_ordered, file = "./mm10_scATAC_GRN.rds")